# Coleta de Dados da Rede Algorand

    - Importando bibliotecas:

In [3]:
from matplotlib import pyplot as plt
from lib2to3.pytree import convert
from ast import literal_eval
from pathlib import Path
from tqdm import tqdm
import seaborn as sns
from ctypes import *
import pandas as pd
import numpy as np
import requests
import struct
import json
import time
import math

--------------------------------------------------------------------------------------

### Coleta de Transações:

    - Classificação dos atributos:


- id:              Hash da transação;
- current-round:   Numero do bloco atual;
- confirmed-round: Numero do bloco;
- round-time:      Timestamp do bloco;
- type:            Tipo da transação;
- fee:             Taxas da transação -> fee = MAX(current_fee_per_byte * len(tnx_in_bytes), min_fee);

In [10]:
#? Inicializando objetos path;
file_obj_1 = Path('../data/transactions/algo_transactions_8.csv')
file_obj_2 = Path('../data/transactions/atual_block.txt')

#? Inicialização do dataframe;
if file_obj_1.is_file():
    df = pd.read_csv('../data/transactions/algo_transactions_8.csv')
else:
    df = pd.DataFrame()

#? Inicialização do número do último bloco coletado;
if file_obj_2.is_file():
    with open("../data/transactions/atual_block.txt", "r") as arq:
        number = arq.read()
else:
    number = 0

In [ ]:
for i in tqdm(range(300)):
    round = str(number)
    url = 'https://algoindexer.algoexplorerapi.io/v2/transactions?limit=8000&min-round=' + round

    block = requests.get(url)
    block = block.json()
    
    tx = block['transactions']
    tx = pd.DataFrame(tx)

    if df.shape[0] > 0:
        df = pd.concat([df, tx])
        number = df['confirmed-round'].iloc[-1]
    else:
        df = tx
        number = df['confirmed-round'].iloc[-1]

with open('../data/transactions/atual_block.txt', 'w') as arquivo:
        arquivo.write(str(number))

print('Número de transações: ' + str(df.shape[0]))
print('Bloco atual: ' + str(number))

In [13]:
#? Salvar dataframe com transações;
df.to_csv('../data/transactions/algo_transactions_8.csv', index=False)

--------------------------------------------------------------------------------------

### Coleta de Contas: